In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import math
import pickle
import numpy as np
from keras_preprocessing.sequence import pad_sequences

file = open("/content/drive/MyDrive/MyModel/MyModel.obj",'rb')
loadedModel = pickle.load(file)
file.close()

print(loadedModel.summary())

file = open("/content/drive/MyDrive/MyModel/MyTokenizer.obj",'rb')
loadedtokenizer = pickle.load(file)
file.close()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 39, 39)            1742988   
                                                                 
 lstm (LSTM)                 (None, 39, 256)           303104    
                                                                 
 dropout (Dropout)           (None, 39, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 44692)             5

In [7]:
with open("/content/test.txt", 'r') as f:
    test_data = [l.strip() for l in f.readlines()]

test_data = ["<s> "+sentence.lower()+" </s>" for sentence in test_data]

In [8]:
train_len = 40
test_text_sequences = []

for sentence in test_data:
    test_tokens = sentence.split()

    if len(test_tokens) > train_len:
        for i in range(train_len, len(test_tokens)):
            seq = test_tokens[i-train_len:i]
            test_text_sequences.append(seq)
    else:
        seq = test_tokens[:]
        test_text_sequences.append(seq)

In [9]:
# for i in (mdl.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
#     pred_word = tokenizer.index_word[i]
#     print("Next word suggestion:",pred_word)

test_encoded_sequences = []
test_text_sequences_size = len(test_text_sequences)
for i in range(test_text_sequences_size):
    encoded_text = loadedtokenizer.texts_to_sequences([test_text_sequences[i]])[0]
    pad_encoded = pad_sequences([encoded_text], maxlen=train_len, truncating='pre')
    test_encoded_sequences.append(pad_encoded[0])

np_test_encoded_sequences = np.empty([len(test_encoded_sequences), train_len], dtype='int32')
for i in range(len(test_encoded_sequences)):
    np_test_encoded_sequences[i] = test_encoded_sequences[i]

test_inputs = np_test_encoded_sequences[:, :-1]
test_set = test_inputs.shape[0]
seq_len = test_inputs.shape[1]
test_targets = np_test_encoded_sequences[:,-1]
# print(test_targets.shape)

In [25]:
total_test_rows = test_inputs.shape[0]

cross_entropy_loss = 0

for i in range(total_test_rows):
    t_reshape = test_inputs[i].reshape(1,seq_len)
    prob = loadedModel.predict(t_reshape)[0]
    if prob[test_targets[i]] <= 0:
      print(prob[test_targets[i]])
    else:
      cross_entropy_loss -= math.log(prob[test_targets[i]])
    print(i, cross_entropy_loss)

Streaming output truncated to the last 5000 lines.
26557 217248.22294433083
26558 217254.89514254988
26559 217256.13778184215
26560 217262.2777733569
26561 217269.09662804363
26562 217274.99170063552
26563 217279.80761516772
26564 217279.80761516772
26565 217279.80761516772
26566 217279.80761516772
26567 217279.80761516772
26568 217279.80761516772
26569 217279.80761516772
26570 217279.80761516772
26571 217279.80761516772
26572 217279.80761516772
26573 217279.80761516772
26574 217279.80761516772
26575 217279.80761516772
26576 217279.80761516772
26577 217279.80761516772
26578 217279.80761516772
26579 217279.80761516772
26580 217279.80761516772
26581 217282.01046044932
26582 217287.34406963867
26583 217289.3609504556
26584 217295.2489501495
26585 217303.06491817234
26586 217307.74765156157
26587 217314.382504766
26588 217322.13815963446
26589 217330.23002164808
26590 217336.81654630063
26591 217336.81654630063
26592 217336.81654630063
26593 217342.5666303156
26594 217346.28416875753
26595

In [28]:
cross_entropy_loss = cross_entropy_loss / test_set
# print(cross_entropy_loss)
ppl = math.pow(2, cross_entropy_loss)

In [29]:
print("The Perplexity of the model on the test set came out to be: ", ppl)

The Perplexity of the model on the test set came out to be:  1.000256711978212
